In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_09_01'
posner_id ='pos1a'
posner_id_b ='pos1b'

IMPORTED MODULES today: 2023-07-17 21:53:00.980561


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_09_01/m_09_01_pos1a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '1a' in file: m_09_01/09_1a_posner_task_2023-05-24_19h15.27.796/09_1a_posner_task_2023-05-24_19h15.27.796.csv
Found '1b' in file: m_09_01/09_1b_posner_task_2023-05-24_20h13.38.774/09_1b_posner_task_2023-05-24_20h13.38.774.csv
********** 1a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Cropping annotations 2023-05-24 19:17:46.020027+00:00 - 2023-05-24 19:31:47.780027+00:00
  [0] Keeping  (2023-05-24 19:17:46.020027+00:00 - 2023-05-24 19:17:46.020027+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-24 19:19:03.524027+00:00 - 2023-05-24 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [371] Keeping  (2023-05-24 19:30:03.112027+00:00 - 2023-05-24 19:30:03.112027+00:00 -> 737.092 - 737.092)
  [372] Keeping  (2023-05-24 19:30:05.642027+00:00 - 2023-05-24 19:30:05.642027+00:00 -> 739.622 - 739.622)
  [373] Keeping  (2023-05-24 19:30:05.742027+00:00 - 2023-05-24 19:30:05.742027+00:00 -> 739.722 - 739.722)
  [374] Keeping  (2023-05-24 19:30:08.112027+00:00 - 2023-05-24 19:30:08.112027+00:00 -> 742.092 - 742.092)
  [375] Keeping  (2023-05-24 19:30:09.612027+00:00 - 2023-05-24 19:30:09.612027+00:00 -> 743.592 - 743.592)
  [376] Keeping  (2023-05-24 19:30:12.462027+00:00 - 2023-05-24 19:30:12.462027+00:00 -> 746.442 - 746.442)
  [377] Keeping  (2023-05-24 19:30:12.562027+00:00 - 2023-05-24 19:30:12.562027+00:00 -> 746.542 - 746.542)
  [378] Keeping  (2023-05-24 19:30:14.613027+00:00 - 2023-05-24 19:30:14.613027+00:00 -> 748.593 - 748.593)
  [379] Keeping  (2023-05-24 19:30:16.112027+00:00 - 2023-05-24 19:30:16.112027+00:00 -> 750.092 - 750.092)
  [380] Keeping  (2023-05-24

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
414 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 414 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 39.5s.


Method,fastica
Fit,63 iterations on raw data (699341 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [7]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [6]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=699341
    Range : 104849 ... 804189 =    104.849 ...   804.189 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


In [8]:
print(ica.exclude)

[48, 21, 32, 33, 0, 1, 3, 14, 10, 19]


In [9]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 10 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [10]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_09_01_pos1a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_01

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_01/m_09_01_pos1a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [11]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_01
├── m_09_01_pos1a.eeg
├── m_09_01_pos1a.vhdr
└── m_09_01_pos1a.vmrk

1 directory, 3 files


In [12]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [13]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_09_01/m_09_01_pos1b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '1a' in file: m_09_01/09_1a_posner_task_2023-05-24_19h15.27.796/09_1a_posner_task_2023-05-24_19h15.27.796.csv
Found '1b' in file: m_09_01/09_1b_posner_task_2023-05-24_20h13.38.774/09_1b_posner_task_2023-05-24_20h13.38.774.csv
********** 1b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-05-24 20:15:51.206851+00:00 - 2023-05-24 20:30:33.946851+00:00
  [0] Keeping  (2023-05-24 20:15:51.206851+00:00 - 2023-05-24 20:15:51.206851+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-24 20:16:24.131851+00:00 - 2023-05-24 20:16:24.131851+00:00 -> 32.925 - 32.925)
  [2] Keeping  (2023-05-24 

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [405] Keeping  (2023-05-24 20:28:47.779851+00:00 - 2023-05-24 20:28:47.779851+00:00 -> 776.573 - 776.573)
  [406] Keeping  (2023-05-24 20:28:47.879851+00:00 - 2023-05-24 20:28:47.879851+00:00 -> 776.673 - 776.673)
  [407] Keeping  (2023-05-24 20:28:49.890851+00:00 - 2023-05-24 20:28:49.890851+00:00 -> 778.684 - 778.684)
  [408] Keeping  (2023-05-24 20:28:51.390851+00:00 - 2023-05-24 20:28:51.390851+00:00 -> 780.184 - 780.184)
  [409] Keeping  (2023-05-24 20:28:54.420851+00:00 - 2023-05-24 20:28:54.420851+00:00 -> 783.214 - 783.214)
  [410] Keeping  (2023-05-24 20:28:54.519851+00:00 - 2023-05-24 20:28:54.519851+00:00 -> 783.313 - 783.313)
  [411] Keeping  (2023-05-24 20:28:56.400851+00:00 - 2023-05-24 20:28:56.400851+00:00 -> 785.194 - 785.194)
Cropping complete (kept 412)


In [14]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
412 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 412 events and 701 original time points ...
106 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.5s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [15]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-05-24 20:17:16.195851+00:00 - 2023-05-24 20:26:11.335851+00:00
  [0] Keeping  (2023-05-24 20:17:16.195851+00:00 - 2023-05-24 20:17:16.195851+00:00 -> 84.989 - 84.989)
  [1] Keeping  (2023-05-24 20:17:17.695851+00:00 - 2023-05-24 20:17:17.695851+00:00 -> 86.489 - 86.489)
  [2] Keeping  (2023-05-24 20:17:21.105851+00:00 - 2023-05-24 20:17:21.105851+00:00 -> 89.899 - 89.899)
  [3] Keeping  (2023-05-24 20:17:21.205851+00:00 - 2023-05-24 20:17:21.205851+00:00 -> 89.999 - 89.999)
  [4] Keeping  (2023-05-24 20:17:22.695851+00:00 - 2023-05-24 20:17:22.695851+00:00 -> 91.489 - 91.489)
  [5] Keeping  (2023-05-24 20:17:24.195851+00:00 - 2023-05-24 20:17:24.195851+00:00 -> 92.989 - 92.989)
  [6] Keeping  (2023-05-24 20:17:27.955851+00:00 - 2023-05-24 20:17:27.955851+00:00 -> 96.749 - 96.749)
  [7] Keeping  (2023-05-24 20:17:28.055851+00:00 - 2023-05-24 20:17

Method,fastica
Fit,98 iterations on raw data (535140 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [16]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=535140
    Range : 84989 ... 620128 =     84.989 ...   620.128 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
267 matching events found
No baseline correction applied
0 projection items ac

In [17]:
ica_b.exclude

[42, 49, 44, 43, 30, 29, 33, 0, 8, 9, 17, 15, 18, 11]

In [18]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 14 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [19]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_01/m_09_01_pos1b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [20]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_01
├── m_09_01_pos1a.eeg
├── m_09_01_pos1a.vhdr
├── m_09_01_pos1a.vmrk
├── m_09_01_pos1b.eeg
├── m_09_01_pos1b.vhdr
└── m_09_01_pos1b.vmrk

1 directory, 6 files


# ...

In [21]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b